In [ ]:
import requests
from selenium import webdriver
import time
import csv
from PIL import Image
import wordcloud
import numpy as np
import jieba
import pandas as pd
import pyecharts
import matplotlib 
from wordcloud import WordCloud, ImageColorGenerator  
import matplotlib.pyplot as plt 



In [ ]:
def get_top_answer():
    browser = webdriver.Chrome()
    browser.get("https://www.zhihu.com/topic/19936951/top-answers")
    time.sleep(5)
 
    for i in range(10):
        browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        time.sleep(3)
        
    divs = browser.find_elements_by_css_selector("div.List-item.TopicFeedItem")

    for div in divs:
        title_info = div.find_element_by_css_selector("h2.ContentItem-title").find_element_by_css_selector("a")
        url = title_info.get_attribute("href")
        title = title_info.text
        count = button.text.split(" ")[1]
        content = get_top_content(url)
 
        with open('知乎REITs.csv', 'a', newline='',encoding="utf-8") as csvfile:
            mywriter = csv.writer(csvfile)
            mywriter.writerow([title,count,content])

In [ ]:
def get_top_content(url):
    browser = webdriver.Chrome()
    browser.get(url)
    time.sleep(3)
    content =""
    try:
        ps = browser.find_element_by_css_selector("div.RichText.ztext.Post-RichText").find_elements_by_css_selector("p")
    except Exception as e:
        ps = browser.find_element_by_css_selector("span.RichText.ztext.CopyrightRichText-richText").find_elements_by_css_selector("p")
    for p in ps:
        content += p.text
    browser.quit()
    return content

In [ ]:
with open("知乎REITs.csv", 'w', newline='',encoding="utf-8") as csvfile:
        mywriter = csv.writer(csvfile)
        header = ["titile", "likes","content"]
        mywriter.writerow(header)
get_top_answer()

In [ ]:
df = pd.read_csv('知乎REITs.csv')

In [ ]:
def cut_content(df):
    all_content = df.content.tolist()
    #print(all_content)
    one_content = ''
    for content in all_content :
        #print(content)
        if type(content) == float:
            continue
        one_content +=  content
        
    words = jieba.cut(one_content)
    #set stopwords
    filepath = 'stopwords.txt'
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]   
    stopwords.extend(['一种','and', 'you', '超过'])
    processed_word_list = [] 
    for word in words:
        if word not in stopwords and len(word)>1: #remove single word
            processed_word_list.append(word)
    return processed_word_list

In [ ]:
processed_word_list = cut_content(df)
print(processed_word_list)

In [ ]:
def word_cloud(processed_word_list, rank=100):
    word_count100 = pd.Series(processed_word_list).value_counts().sort_values(ascending=False)[:rank]
    word_count100_dict = word_count100.to_dict()
    print(word_count100_dict)
    result = " ".join(word_count100_dict)  
    
    wc = WordCloud(
        font_path="simhei.ttf",
        background_color='white',   
        max_font_size=50,   #最大字号
        
    )
    wc.generate(result)
    plt.figure(figsize=(12,16))
    plt.imshow(wc, interpolation='bilinear')
    plt.axis("off")
    plt.title('word frequency top '+str(rank)+' tag cloud', loc='Center', fontsize=26)
    plt.show()
    return plt.show()

word_cloud(processed_word_list)

